In [ ]:
import torch
import transformers

from collections import OrderedDict

import sys
sys.path.insert(0, '..')

from roberta_model import RobertaForSequenceClassification, RobertaConfig
from decompose_roberta import RobertaForSequenceClassificationDecomposed, RobertaForMaskedLMDecomposed


In [2]:
model_hf = transformers.AutoModelForSequenceClassification.from_pretrained("roberta-base")
tokenizer = transformers.AutoTokenizer.from_pretrained("roberta-base")
model_hf.eval()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [3]:
model = RobertaForSequenceClassification(RobertaConfig.from_dict(model_hf.config.to_dict()))


In [4]:
new_state_dict = OrderedDict()
for key, value in model_hf.state_dict().items():
    # rename weight values in state_dict from roberta to bert
    new_key = key.replace("roberta", "bert")
    new_key = new_key.replace(
        "classifier.dense", "bert.pooler.dense").replace(
            "classifier.out_proj", "classifier")
    # ref : https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html
    new_key = new_key.replace("LayerNorm.weight",
                           "LayerNorm.gamma").replace("LayerNorm.bias",
                                                       "LayerNorm.beta")
    new_key = new_key.replace("lm_head.layer_norm.weight",
                           "cls.predictions.transform.LayerNorm.gamma").replace("lm_head.layer_norm.bias",
                                                       "cls.predictions.transform.LayerNorm.beta")
    new_key = new_key.replace("lm_head.decoder", "cls.predictions.decoder")
    new_key = new_key.replace("lm_head.dense", "cls.predictions.transform.dense")
    new_key = new_key.replace("cls.predictions.decoder.bias", "cls.predictions.transform.dense")
    new_key = new_key.replace("lm_head.bias", "cls.predictions.bias")
    
    new_state_dict[new_key] = value
    
model.load_state_dict(new_state_dict)
model.eval()


RobertaForSequenceClassification(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12)
              (

In [5]:
torch.allclose(
    model_hf.roberta.encoder.layer[0].attention.self.query.weight,
    model.bert.encoder.layer[0].attention.self.query.weight,
    atol=1e-5)


True

In [19]:
# tokenizer.pad_token = tokenizer.eos_token
# model_hf.config.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

inputs = tokenizer(["Hello, my dog is cute", "Paris is the capital of France"], return_tensors="pt",
                   truncation=True, padding="max_length", max_length=128
                   )

inputs["input_ids"] = inputs["input_ids"]
model_hf.eval()

with torch.no_grad():
    logits_hf = model_hf(**inputs)

logits_hf.logits


tensor([[-0.0956,  0.0408],
        [-0.0959,  0.0360]])

In [20]:
with torch.no_grad():
    logits = model(**inputs)

logits


tensor([[-0.0956,  0.0408],
        [-0.0959,  0.0360]])

In [21]:
model_decomposed = RobertaForSequenceClassificationDecomposed.from_pretrained(
    "roberta-base", num_labels=2, debug=True, num_contributions=2, shapley_include_bias=False)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
beta_mask = (torch.rand(128) > 0.5).to(int)
beta_mask = torch.stack([beta_mask, 1 - beta_mask])
beta_mask = torch.stack([beta_mask] * 2)
print(beta_mask.shape)

with torch.no_grad():
    decomposed_logits = model_decomposed(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        beta_mask=beta_mask)


torch.Size([2, 2, 128])
Roberta Encoder Layer 0 error: 7.053347741979863e-16
Roberta Encoder Layer 1 error: 1.3655197442356153e-15
Roberta Encoder Layer 2 error: 1.533670197060587e-15
Roberta Encoder Layer 3 error: 1.7615183232214401e-15
Roberta Encoder Layer 4 error: 2.260770988858441e-15
Roberta Encoder Layer 5 error: 2.4090850936261123e-15
Roberta Encoder Layer 6 error: 3.6008283389858066e-15
Roberta Encoder Layer 7 error: 6.1220706944279405e-15
Roberta Encoder Layer 8 error: 1.3595941591552308e-14
Roberta Encoder Layer 9 error: 2.939504930914178e-14
Roberta Encoder Layer 10 error: 4.8713760772716e-14
Roberta Encoder Layer 11 error: 2.1991366542082924e-14

Pooled output error:  2.1202301854085153e-15

Roberta Classifier logits error:  6.660253945578454e-16


In [28]:
decompose


Roberta Encoder Layer 0 error: 7.053347741979863e-16
Roberta Encoder Layer 1 error: 1.3655197442356153e-15
Roberta Encoder Layer 2 error: 1.533670197060587e-15
Roberta Encoder Layer 3 error: 1.7615183232214401e-15
Roberta Encoder Layer 4 error: 2.260770988858441e-15
Roberta Encoder Layer 5 error: 2.4090850936261123e-15
Roberta Encoder Layer 6 error: 3.6008283389858066e-15
Roberta Encoder Layer 7 error: 6.1220706944279405e-15
Roberta Encoder Layer 8 error: 1.3595941591552308e-14
Roberta Encoder Layer 9 error: 2.939504930914178e-14
Roberta Encoder Layer 10 error: 4.8713760772716e-14
Roberta Encoder Layer 11 error: 2.1991366542082924e-14


torch.Size([3, 2, 128, 768])

In [10]:
model_decomposed = RobertaForSequenceClassificationDecomposed.from_pretrained(
    "roberta-base", num_labels=2, debug=True, num_contributions=2,
    shapley_include_bias=True)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
beta_mask = (torch.rand(128) > 0.5).to(int)
beta_mask = torch.stack([beta_mask, 1 - beta_mask])
beta_mask = torch.stack([beta_mask] * 2)
print(beta_mask.shape)

with torch.no_grad():
    decomposed_logits = model_decomposed(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        beta_mask=beta_mask)


torch.Size([2, 2, 128])
Roberta Encoder Layer 0 error: 9.45152613001158e-16
Roberta Encoder Layer 1 error: 4.048656048689765e-15
Roberta Encoder Layer 2 error: 2.4831058167188317e-14
Roberta Encoder Layer 3 error: 3.1059510149682104e-13
Roberta Encoder Layer 4 error: 4.127979700060342e-12
Roberta Encoder Layer 5 error: 4.3145703932436284e-11
Roberta Encoder Layer 6 error: 2.449237399272597e-10
Roberta Encoder Layer 7 error: 1.1569460553915231e-09
Roberta Encoder Layer 8 error: 5.917498722587327e-09
Roberta Encoder Layer 9 error: 2.4135236333255104e-08
Roberta Encoder Layer 10 error: 4.5486368738502e-08
Roberta Encoder Layer 11 error: 2.1228192026963555e-08

Pooled output error:  2.6192674314963653e-09

Roberta Classifier logits error:  2.135024802785157e-09


In [12]:
inputs = tokenizer(["Hello, did you know that the <mask> of France is <mask>?", "Hello, did you know that the <mask> of France is <mask>?"], return_tensors="pt",
                   truncation=True, padding="max_length", max_length=128
                   )


In [13]:
model_decomposed = RobertaForMaskedLMDecomposed.from_pretrained(
    "roberta-base", debug=True, num_contributions=2,
    shapley_include_bias=False)


In [16]:
beta_mask = (torch.rand(128) > 0.5).to(int)
beta_mask = torch.stack([beta_mask, 1 - beta_mask])
beta_mask = torch.stack([beta_mask] * 2)
print(beta_mask.shape)

with torch.no_grad():
    decomposed_logits = model_decomposed(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        beta_mask=beta_mask)


torch.Size([2, 2, 128])

Roberta LM predictions error:  4.75799996183548e-12


In [17]:
model_decomposed = RobertaForMaskedLMDecomposed.from_pretrained(
    "roberta-base", debug=True, num_contributions=2,
    shapley_include_bias=True)


In [18]:
beta_mask = (torch.rand(128) > 0.5).to(int)
beta_mask = torch.stack([beta_mask, 1 - beta_mask])
beta_mask = torch.stack([beta_mask] * 2)
print(beta_mask.shape)

with torch.no_grad():
    decomposed_logits = model_decomposed(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        beta_mask=beta_mask)


torch.Size([2, 2, 128])

Roberta LM predictions error:  2.864429651130968e-07
